In [2]:
def make_is_bad_zero(Xy_subset, min_interval=48, summer_start=3000, summer_end=7500):
    """Helper routine for 'find_bad_zeros'.
    
    This operates upon a single dataframe produced by 'groupby'. We expect an 
    additional column 'meter_id' which is a duplicate of 'meter' because groupby 
    eliminates the original one."""
    meter = Xy_subset.meter_id.iloc[0]
    is_zero = Xy_subset.meter_reading == 0
    if meter == 0:
        # Electrical meters should never be zero. Keep all zero-readings in this table so that
        # they will all be dropped in the train set.
        return is_zero

    transitions = (is_zero != is_zero.shift(1))
    all_sequence_ids = transitions.cumsum()
    ids = all_sequence_ids[is_zero].rename("ids")
    if meter in [2, 3]:
        # It's normal for steam and hotwater to be turned off during the summer
        keep = set(ids[(Xy_subset.timestamp < summer_start) |
                       (Xy_subset.timestamp > summer_end)].unique())
        is_bad = ids.isin(keep) & (ids.map(ids.value_counts()) >= min_interval)
    elif meter == 1:
        time_ids = ids.to_frame().join(Xy_subset.timestamp).set_index("timestamp").ids
        is_bad = ids.map(ids.value_counts()) >= min_interval

        # Cold water may be turned off during the winter
        jan_id = time_ids.get(0, False)
        dec_id = time_ids.get(8283, False)
        if (jan_id and dec_id and jan_id == time_ids.get(500, False) and
                dec_id == time_ids.get(8783, False)):
            is_bad = is_bad & (~(ids.isin(set([jan_id, dec_id]))))
    else:
        raise Exception(f"Unexpected meter type: {meter}")

    result = is_zero.copy()
    result.update(is_bad)
    return result

def find_bad_zeros(X, y):
    """Returns an Index object containing only the rows which should be deleted."""
    Xy = X.assign(meter_reading=y, meter_id=X.meter)
    is_bad_zero = Xy.groupby(["building_id", "meter"]).apply(make_is_bad_zero)
    return is_bad_zero[is_bad_zero].index.droplevel([0, 1])

def find_bad_sitezero(X):
    """Returns indices of bad rows from the early days of Site 0 (UCF)."""
    return X[(X.timestamp < 3378) & (X.site_id == 0) & (X.meter == 0)].index

def find_bad_building1099(X, y):
    """Returns indices of bad rows (with absurdly high readings) from building 1099."""
    return X[(X.building_id == 1099) & (X.meter == 2) & (y > 3e4)].index

def find_bad_rows(X, y):
    return find_bad_zeros(X, y).union(find_bad_sitezero(X)).union(find_bad_building1099(X, y))

import pandas as pd
import numpy as np
import os
import warnings


from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.metrics import mean_squared_log_error

pd.set_option("max_columns", 500)

def input_file(file):
    path = f"{file}"
    if not os.path.exists(path): return path + ".gz"
    return path

def compress_dataframe(df):
    result = df.copy()
    for col in result.columns:
        col_data = result[col]
        dn = col_data.dtype.name
        if dn == "object":
            result[col] = pd.to_numeric(col_data.astype("category").cat.codes, downcast="integer")
        elif dn == "bool":
            result[col] = col_data.astype("int8")
        elif dn.startswith("int") or (col_data.round() == col_data).all():
            result[col] = pd.to_numeric(col_data, downcast="integer")
        else:
            result[col] = pd.to_numeric(col_data, downcast='float')
    return result

def read_train():
    df = pd.read_csv(input_file("train.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    return compress_dataframe(df)

def read_building_metadata():
    return compress_dataframe(pd.read_csv(
        input_file("building_metadata.csv")).fillna(-1)).set_index("building_id")

site_GMT_offsets = [-5, 0, -7, -5, -8, 0, -5, -5, -5, -6, -7, -5, 0, -6, -5, -5]

def read_weather_train(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    df = pd.read_csv(input_file("weather_train.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    if fix_timestamps:
        GMT_offset_map = {site: offset for site, offset in enumerate(site_GMT_offsets)}
        df.timestamp = df.timestamp + df.site_id.map(GMT_offset_map)
    if interpolate_na:
        site_dfs = []
        for site_id in df.site_id.unique():
            # Make sure that we include all possible hours so that we can interpolate evenly
            site_df = df[df.site_id == site_id].set_index("timestamp").reindex(range(8784))
            site_df.site_id = site_id
            for col in [c for c in site_df.columns if c != "site_id"]:
                if add_na_indicators: site_df[f"had_{col}"] = ~site_df[col].isna()
                site_df[col] = site_df[col].interpolate(limit_direction='both', method='linear')
                # Some sites are completely missing some columns, so use this fallback
                site_df[col] = site_df[col].fillna(df[col].median())
            site_dfs.append(site_df)
        df = pd.concat(site_dfs).reset_index()  # make timestamp back into a regular column
    elif add_na_indicators:
        for col in df.columns:
            if df[col].isna().any(): df[f"had_{col}"] = ~df[col].isna()
    return compress_dataframe(df).set_index(["site_id", "timestamp"])

def combined_train_data(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    Xy = compress_dataframe(read_train().join(read_building_metadata(), on="building_id").join(
        read_weather_train(fix_timestamps, interpolate_na, add_na_indicators),
        on=["site_id", "timestamp"]).fillna(-1))
    return Xy.drop(columns=["meter_reading"]), Xy.meter_reading

def _add_time_features(X):
    return X.assign(tm_day_of_week=((X.timestamp // 24) % 7), tm_hour_of_day=(X.timestamp % 24))

class CatSplitRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, model, col):
        self.model = model
        self.col = col

    def fit(self, X, y):
        self.fitted = {}
        importances = []
        for val in X[self.col].unique():
            X1 = X[X[self.col] == val].drop(columns=[self.col])
            self.fitted[val] = clone(self.model).fit(X1, y.reindex_like(X1))
            importances.append(self.fitted[val].feature_importances_)
            del X1
        fi = np.average(importances, axis=0)
        col_index = list(X.columns).index(self.col)
        self.feature_importances_ = [*fi[:col_index], 0, *fi[col_index:]]
        return self

    def predict(self, X):
        result = np.zeros(len(X))
        for val in X[self.col].unique():
            ix = np.nonzero((X[self.col] == val).to_numpy())
            predictions = self.fitted[val].predict(X.iloc[ix].drop(columns=[self.col]))
            result[ix] = predictions
        return result

categorical_columns = [
    "building_id", "meter", "site_id", "primary_use", "had_air_temperature", "had_cloud_coverage",
    "had_dew_temperature", "had_precip_depth_1_hr", "had_sea_level_pressure", "had_wind_direction",
    "had_wind_speed", "tm_day_of_week", "tm_hour_of_day"
]


def read_test():
    df = pd.read_csv(input_file("test.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    return compress_dataframe(df).set_index("row_id")

def read_weather_test(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    df = pd.read_csv(input_file("weather_test.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    if fix_timestamps:
        GMT_offset_map = {site: offset for site, offset in enumerate(site_GMT_offsets)}
        df.timestamp = df.timestamp + df.site_id.map(GMT_offset_map)
    if interpolate_na:
        site_dfs = []
        for site_id in df.site_id.unique():
            # Make sure that we include all possible hours so that we can interpolate evenly
            site_df = df[df.site_id == site_id].set_index("timestamp").reindex(range(8784, 26304))
            site_df.site_id = site_id
            for col in [c for c in site_df.columns if c != "site_id"]:
                if add_na_indicators: site_df[f"had_{col}"] = ~site_df[col].isna()
                site_df[col] = site_df[col].interpolate(limit_direction='both', method='linear')
                # Some sites are completely missing some columns, so use this fallback
                site_df[col] = site_df[col].fillna(df[col].median())
            site_dfs.append(site_df)
        df = pd.concat(site_dfs).reset_index()  # make timestamp back into a regular column
    elif add_na_indicators:
        for col in df.columns:
            if df[col].isna().any(): df[f"had_{col}"] = ~df[col].isna()
    return compress_dataframe(df).set_index(["site_id", "timestamp"])

def combined_test_data(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    X = compress_dataframe(read_test().join(read_building_metadata(), on="building_id").join(
        read_weather_test(fix_timestamps, interpolate_na, add_na_indicators),
        on=["site_id", "timestamp"]).fillna(-1))
    return X

In [3]:
X, y = combined_train_data()

bad_rows = find_bad_rows(X, y)
pd.Series(bad_rows.sort_values()).to_csv("rows_to_drop.csv", header=False, index=False)

In [4]:
X = X.drop(index=bad_rows)
y = y.reindex_like(X)

In [17]:
X.head(100)

,building_id,meter,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,had_air_temperature,had_cloud_coverage,had_dew_temperature,had_precip_depth_1_hr,had_sea_level_pressure,had_wind_direction,had_wind_speed,tm_day_of_week,tm_hour_of_day
103,105,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
104,106,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
105,106,3,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
106,107,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
107,108,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
108,109,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
109,109,3,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
110,110,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
111,111,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
112,112,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0


In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Additional preprocessing


del X, y

In [11]:
dfb = read_building_metadata()

In [5]:

building_columns = ['site_id', 'primary_use', 'square_feet', 'year_built', 'floor_count']


In [16]:
X = compress_dataframe(_add_time_features(X))
X = X.drop(columns="timestamp")  # Raw timestamp doesn't help when prediction
X = X.drop(columns=building_columns)
y = np.log1p(y)

In [26]:
X.to_csv('X.csv')

In [27]:
y.to_csv('y.csv')

C:\conda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [ ]:
meter = 0
building = 1 

,building_id,meter,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,had_air_temperature,had_cloud_coverage,had_dew_temperature,had_precip_depth_1_hr,had_sea_level_pressure,had_wind_direction,had_wind_speed,tm_day_of_week,tm_hour_of_day
103,105,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
104,106,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
105,106,3,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
106,107,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0
107,108,0,3.8,0.0,2.4,0.0,1020.900024,240.0,3.1,1,0,1,0,1,1,1,0,0


In [21]:
dfd = X[(X.meter == 0) & (X.building_id == 1)]

In [25]:
dfd.head()
yd = y[dfd.index].head()


7576723    4.558774
7579070    4.843116
7581416    4.847410
7583763    4.853818
7586109    4.889379
Name: meter_reading, dtype: float32

In [1]:
import pandas as pd

In [3]:
X = pd.read_csv('X.csv')

In [6]:
df103 = X[X.building_id == 103]

In [7]:
df103.head()

,Unnamed: 0,building_id,meter,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,had_air_temperature,had_cloud_coverage,had_dew_temperature,had_precip_depth_1_hr,had_sea_level_pressure,had_wind_direction,had_wind_speed,tm_day_of_week,tm_hour_of_day
2892220,3172402,103,1,16.7,6.0,10.6,0.0,1023.3,0.0,0.0,1,0,1,1,1,1,1,3,9
2894145,3174501,103,1,21.1,6.0,9.4,0.0,1023.2,200.0,2.1,1,0,1,1,1,1,1,3,10
2896073,3176604,103,1,21.7,6.0,8.3,0.0,1023.1,100.0,1.5,1,0,1,1,1,0,1,3,11
2897998,3178706,103,1,22.2,6.0,8.3,0.0,1022.7,0.0,0.0,1,0,1,1,1,1,1,3,12
2899917,3180797,103,1,23.9,6.0,8.3,0.0,1021.6,350.0,2.1,1,1,1,1,1,1,1,3,13


In [8]:
df103.describe()

,Unnamed: 0,building_id,meter,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,had_air_temperature,had_cloud_coverage,had_dew_temperature,had_precip_depth_1_hr,had_sea_level_pressure,had_wind_direction,had_wind_speed,tm_day_of_week,tm_hour_of_day
count,1.266700e+04,12667.0,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000,12667.000000
mean,1.256177e+07,103.0,0.573616,24.580414,4.209994,18.813421,1.462856,1017.819101,147.093234,3.231191,0.999211,0.577011,0.999211,0.999211,0.991790,0.967080,0.999211,2.997158,11.521986
std,4.595472e+06,0.0,0.494570,5.009991,2.203209,5.193929,13.853059,3.702332,115.586631,2.112567,0.028087,0.494053,0.028087,0.028087,0.090242,0.178435,0.028087,2.003705,6.920876
min,3.172402e+06,103.0,0.000000,5.000000,0.000000,-9.400000,-1.000000,992.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.002196e+06,103.0,0.000000,21.700000,2.000000,16.100000,0.000000,1015.600000,50.000000,2.100000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000
50%,1.269385e+07,103.0,1.000000,25.000000,4.000000,20.600000,0.000000,1017.900000,130.000000,3.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,12.000000
75%,1.651510e+07,103.0,1.000000,27.800000,6.000000,22.800000,0.000000,1020.200000,235.000000,4.600000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,18.000000
max,2.021386e+07,103.0,1.000000,36.100000,9.000000,25.600000,343.000000,1028.500000,360.000000,15.400000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,23.000000


In [6]:
X = combined_test_data()
X = compress_dataframe(_add_time_features(X))
X = X.drop(columns="timestamp")  # Raw timestamp doesn't help when prediction
X = X.drop(columns=building_columns)


In [8]:
X_test = X
X_test.head()

,building_id,meter,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,had_air_temperature,had_cloud_coverage,had_dew_temperature,had_precip_depth_1_hr,had_sea_level_pressure,had_wind_direction,had_wind_speed,tm_day_of_week,tm_hour_of_day
row_id,,,,,,,,,,,,,,,,,,
0,0,0,15.6,2.0,12.8,0.0,1022.099976,130.0,2.1,1,1,1,1,1,1,1,2,0
1,1,0,15.6,2.0,12.8,0.0,1022.099976,130.0,2.1,1,1,1,1,1,1,1,2,0
2,2,0,15.6,2.0,12.8,0.0,1022.099976,130.0,2.1,1,1,1,1,1,1,1,2,0
3,3,0,15.6,2.0,12.8,0.0,1022.099976,130.0,2.1,1,1,1,1,1,1,1,2,0
4,4,0,15.6,2.0,12.8,0.0,1022.099976,130.0,2.1,1,1,1,1,1,1,1,2,0


In [9]:
X_test.to_csv('X_test.csv')

In [10]:


del X, X_test

In [ ]:
predictions = pd.DataFrame({
    "row_id": X.index,
    "meter_reading": np.clip(np.expm1(model.predict(X)), 0, None)
})